# Task 1 

### Run cell below if running on Google Colab

In [ ]:
from google.colab import drive
# drive.flush_and_unmount()  # Unmount Google Drive
drive.mount('/content/drive', force_remount=True)

### Create manual prompt for generating write-ups using gpt-4o-mini for each welfare scheme

In [ ]:
import os

# Directory containing the text files
directory = "/content/drive/MyDrive/" + "remaining path of welfare scheme txt file"

# Limit to the first 10 files found in the directory (we had 10 schemes)
file_paths = [os.path.join(directory, file) for file in os.listdir(directory) if file.endswith(".txt")][:10] 

# Dictionary to store each file's content
file_contents = {}

# Read contents of each file
for file_path in file_paths:
    with open(file_path, 'r') as file:
        content = file.read()
        file_name = os.path.basename(file_path)
        file_contents[file_name] = content

# Function to create prompt for each file's content
def generate_prompt(file_name, content):
    prompt = f"""The following text describes a welfare scheme. Based on this information, generate the following write-ups for {file_name}.
    Retain all relevant information from the original text. No significant details should be omitted. Write-ups should be clear, concise, and easily understandable.

Write-up 1: Beneficiary and Problem Statement
    Describe who the beneficiary is and the specific problems or challenges they are currently facing.

Write-up 2: Application Process and Benefits
    Detail the steps required to apply for the scheme and the exact benefits provided upon successful application.

Write-up 3: Outcome and Impact
    Explain the expected outcomes of the scheme and how it will positively impact the beneficiary's life.

Scheme Description:
{content}
"""
    return prompt

# Generate prompts for each file
prompts = {file_name: generate_prompt(file_name, content) for file_name, content in file_contents.items()}

# Display prompts for review or use as input to ChatGPT
for file_name, prompt in prompts.items():
    print(f"Prompt for {file_name}:\n{prompt}\n{'-'*80}\n\n")

# Note: After generating the prompts, you would typically pass each prompt to ChatGPT's API for response.


### Create 3 writeups for each scheme using gpt-4o-mini

In [ ]:
from openai import OpenAI
client = OpenAI(api_key = 'openai_api')

output_directory = "/content/drive/MyDrive/" + "remaining path for outputs for LLM write-ups"
# Ensure the output directory exists
os.makedirs(output_directory, exist_ok=True)

for file_name, prompt in prompts.items():
    # print(f"Prompt for {file_name}:\n{prompt}\n{'-'*80}\n\n")

    completion = client.chat.completions.create(
        model="gpt-4o-mini", # Use better model for better quality prompts
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )
    write_up = completion.choices[0].message.content
    # print(completion.choices[0].message.content)

    # Save the write-up to a new .txt file in the output directory with the same file name
    output_file_path = os.path.join(output_directory, file_name)
    with open(output_file_path, 'w+') as output_file:
        output_file.write(write_up)

    print(f"Write-up for '{file_name}' saved successfully in '{output_directory}'")



### Code to generate summary of write-ups using T5 transformer
#### (Executing the 2 cells below is not necessary)

In [ ]:
!pip install transformers nltk
!pip install evaluate

In [ ]:
import os
from transformers import T5Tokenizer, T5ForConditionalGeneration


input_directory = "/content/drive/MyDrive/" + "remaining path of welfare scheme txt file"
output_directory = "/content/drive/MyDrive/" + "remaining path for outputs for T5 write-ups"
# Ensure the output directory exists
os.makedirs(output_directory, exist_ok=True)

# Load T5 Model and Tokenizer
model_name = "t5-base"  # or "t5-base" for a larger model
model = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name)


for file_name, prompt in prompts.items():
    file_path = input_directory + file_name

    with open(file_path, 'r') as file:
        content = file.read()


    inputs = tokenizer(f"Scheme Description: " + content +
                   """Based on the above information, please provide a summary of 150 words. Retain all relevant information from the original text. """,
                   return_tensors="pt",
                   max_length=2048,
                   truncation=True)
    summary_ids = model.generate(inputs.input_ids, max_length=2048, num_beams=8, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    # print(summary)

    # Save the write-up to a new .txt file in the output directory with the same file name
    output_file_path = os.path.join(output_directory, file_name)
    with open(output_file_path, 'w+') as output_file:
        output_file.write(summary)

    print(f"Write-up for '{file_name}' saved successfully in '{output_directory}'")


### Create 3 image generation prompts for each welfare scheme

In [ ]:
import os
from openai import OpenAI

client = OpenAI(api_key = "openai_api")

output_directory = "/content/drive/MyDrive/" + "remaining path for image prompts"
# Ensure the output directory exists
os.makedirs(output_directory, exist_ok=True)

directory = "/content/drive/MyDrive/NLP/" + "remaining path for outputs for LLM write-ups"
file_paths = [os.path.join(directory, file) for file in os.listdir(directory) if file.endswith(".txt")]

# Initialize the results dictionary
results = []

for file_path in file_paths:
    with open(file_path, 'r') as file:
        content = file.read()
        file_name = os.path.basename(file_path)

    # Remove '.txt' from the file name
    scheme_name = file_name.replace(".txt", "")

    prompt1 = f"""
        Generate an image prompt to depict the 'Problem Statement' for the '{scheme_name}' scheme:\n\n"
        {content}\n\n
        Instructions:\n
        - Show the main character (e.g., 'Raju the farmer' or 'Shweta the housewife', etc, whatever is more appropritae) experiencing the challenges described in the scheme's problem statement, such as financial struggles or resource shortages, etc.\n
        - Use a rural or village setting with elements like small farms, simple houses, or crops to illustrate the environment, according to the scheme's objective.\n
        - Focus on conveying the difficulties faced by beneficiaries before receiving support, such as worried expressions, sparse resources, or modest living conditions, etc.
        """
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that creates image generation prompts."},
            {"role": "user", "content": prompt1}
        ]
    )
    img_prompt1 = completion.choices[0].message.content

    prompt2 = f"""
        Generate an image prompt to illustrate the 'Application Process' for the '{scheme_name}' scheme:\n\n"
        "{content}\n\n"
        "Instructions:\n"
        "- Depict a scene showing the enrollment steps, with the main character (e.g., 'Raju the farmer' or 'Shweta the housewife', etc, whatever is more approprite and mentioned later) at a registration desk or using a digital device.\n
        "- Highlight the required documents, such as Aadhaar card, bank passbook, or application forms, as visible elements in the scene.\n
        "- Include a friendly government official or digital interface to emphasize the assistance provided during the application process.\n
        "- Set the scene in a government office or rural setting, focusing on the action of registration and the accessibility of the process.
        Here is the previous image prompt, ensure consistency of characters and story:
        {img_prompt1}
        """

    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that creates image generation prompts."},
            {"role": "user", "content": prompt2}
        ]
    )
    img_prompt2 = completion.choices[0].message.content

    prompt3 = f"""
        Generate an image prompt for the '{scheme_name}' scheme:\n\n"
        f"{content}\n\n"
        Instructions:\n
        - Use a rural/appropriate setting and portray characters benefiting from a government support scheme.
        - Include symbols of agricultural aid or community support, illustrating the scheme’s purpose.
        Here are the previous prompts, ensure consistency of characters and story:
        {img_prompt1}
        {img_prompt2}
        """

    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that creates image generation prompts."},
            {"role": "user", "content": prompt3}
        ]
    )
    img_prompt3 = completion.choices[0].message.content

    results.append({
        'scheme_name': scheme_name,
        'prompt1': img_prompt1,
        'prompt2': img_prompt2,
        'prompt3': img_prompt3
    })

    prompt1_path = scheme_name+"_1.txt"
    prompt2_path = scheme_name+"_2.txt"
    prompt3_path = scheme_name+"_3.txt"
    output_file_path = os.path.join(output_directory, prompt1_path)
    with open(output_file_path, 'w+') as output_file:
        output_file.write(img_prompt1)

    output_file_path = os.path.join(output_directory, prompt2_path)
    with open(output_file_path, 'w+') as output_file:
        output_file.write(img_prompt2)

    output_file_path = os.path.join(output_directory, prompt3_path)
    with open(output_file_path, 'w+') as output_file:
        output_file.write(img_prompt3)

    print(f"Image prompts for '{file_name}' saved successfully in '{output_directory}'")


# Once the loop finishes, write the results dictionary to a text file
results_file_path = "/content/drive/MyDrive/" + "relative path" + "results_final.txt"
with open(results_file_path, 'w') as results_file:
    results_file.write(str(results))

print(f"Results written to '{results_file_path}'")

### Code to check quality of image prompts
#### (Executing the 2 cells below is not necessary)

In [ ]:
!pip install textstat
import os
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoTokenizer, AutoModel
import torch
import spacy
import textstat
from glob import glob

In [ ]:
# Load a transformer model for embeddings
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

def read_prompts(file_paths):
    prompts = []
    for path in file_paths:
        with open(path, "r") as file:
            prompts.append(file.read().strip())
    return prompts

def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).detach().numpy().reshape(1, -1)

def calculate_cosine_similarity(prompts):
    embeddings = [get_embedding(prompt) for prompt in prompts]
    sim1 = cosine_similarity(embeddings[0], embeddings[1])
    sim2 = cosine_similarity(embeddings[0], embeddings[2])
    sim3 = cosine_similarity(embeddings[1], embeddings[2])
    all_similarities = np.array([sim1[0][0], sim2[0][0], sim3[0][0]])
    avg_similarity = np.mean(all_similarities)
    return avg_similarity

def calculate_token_lengths(prompts):
    return [len(prompt.split()) for prompt in prompts]

def calculate_lexical_diversity(prompts):
    return [len(set(prompt.split())) / len(prompt.split()) for prompt in prompts]

def calculate_readability_scores(prompts):
    flesch_scores = [textstat.flesch_reading_ease(prompt) for prompt in prompts]
    fog_scores = [textstat.gunning_fog(prompt) for prompt in prompts]
    return flesch_scores, fog_scores

def evaluate_prompts_for_scheme(scheme_name, file_paths):
    prompts = read_prompts(file_paths)

    # Consistency metrics
    avg_cosine_similarity = calculate_cosine_similarity(prompts)
    token_lengths = calculate_token_lengths(prompts)
    lexical_diversity = calculate_lexical_diversity(prompts)
    flesch_scores, fog_scores = calculate_readability_scores(prompts)

    print(f"Metrics for Scheme: {scheme_name}")
    print(f"Average Cosine Similarity: {avg_cosine_similarity:.4f}")
    print("Token Lengths:", token_lengths)
    print("Lexical Diversity:", [f"{diversity:.4f}" for diversity in lexical_diversity])
    print("Readability Scores (Flesch Reading Ease):", flesch_scores)
    print("Readability Scores (Gunning Fog):", fog_scores)
    print("-" * 60)

def main(directory_path):
    file_paths = glob(os.path.join(directory_path, "*.txt"))
    scheme_names = set(os.path.basename(f).split('_')[0] for f in file_paths)

    for scheme_name in scheme_names:
        problem_file = os.path.join(directory_path, f"{scheme_name}_1.txt")
        application_file = os.path.join(directory_path, f"{scheme_name}_2.txt")
        impact_file = os.path.join(directory_path, f"{scheme_name}_3.txt")

        # Check if all three files exist
        if os.path.exists(problem_file) and os.path.exists(application_file) and os.path.exists(impact_file):
            evaluate_prompts_for_scheme(scheme_name, [problem_file, application_file, impact_file])
        else:
            print(f"Warning: Not all prompt files found for scheme '{scheme_name}'.")

# Define the directory where all prompt files are stored
directory_path = "/content/drive/MyDrive/" + "remaining path of image prompts"

# Run the evaluation
main(directory_path)

### Generate images using dall-e-3

In [ ]:
import os
import requests
from openai import OpenAI

# Input and output directories
input_directory = "/content/drive/MyDrive/" + "remaining path for image prompts"
output_directory = "/content/drive/MyDrive/" + "remaining path for image outputs"

# Ensure the output directory exists
os.makedirs(output_directory, exist_ok=True)

# Collect paths of all .txt files in the input directory
file_paths = [os.path.join(input_directory, file) for file in os.listdir(input_directory) if file.endswith(".txt")]

# Initialize OpenAI client with API key
client = OpenAI(api_key='openai_api')

# Loop through each file and generate an image
for file_path in file_paths:
    # Read the file contents as prompt
    with open(file_path, 'r') as file:
        prompt = file.read().strip()  # Remove any leading/trailing whitespace

    # Generate the image
    response = client.images.generate(
        model="dall-e-3",
        prompt=prompt,
        size="1024x1024",
        quality="standard",
        n=1,
    )

    # Retrieve the image URL from the response
    image_url = response.data[0].url

    # Download the image and save it to the output directory
    image_data = requests.get(image_url).content
    image_filename = os.path.splitext(os.path.basename(file_path))[0] + ".png"
    image_path = os.path.join(output_directory, image_filename)

    # Save the image
    with open(image_path, "wb") as image_file:
        image_file.write(image_data)

    print(f"Image saved to {image_path}")


# Assigment Part 2 - Pipeline
## Iterative improvement of image relevance

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from google.colab import auth

PROJECT_ID = "google_cloud_project"

# login using the email ID in the google cloud project
auth.authenticate_user(project_id=PROJECT_ID)

### Code to generate better image prompts using previously generated LLM write-up
#### (Commented prompt is older prompt)

In [ ]:
from openai import OpenAI

client = OpenAI(api_key = "openai_api")

def get_basic_prompts(filename, file_path):
    with open(file_path, 'r') as file:
        content = file.read()
        file_name = os.path.basename(file_path)

    # Remove '.txt' from the file name
    scheme_name = file_name.replace(".txt", "")

    description_prompt = f"""Generate a detailed description of all characters, that should be used for creating posters for {scheme_name}.
    It should have 2-3 main characters at max. Explicitly write all features describing the outwards appearance of each character.
    Also mention the art style of the poster in definite art forms liek realistic, manga, cartoon, etc.
    Do not specify background or facial expressions of characters, because that will be determined according to the scene.
    Please write only the clear and concise answer."""

    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that helps in creating image generation prompts."},
            {"role": "user", "content": description_prompt}
        ]
    )
    description = completion.choices[0].message.content

    prompt1 = f"""
        Generate a short image prompt to depict the 'Problem Statement' for the '{scheme_name}':\n\n"
        {content}\n\n
        Instructions:\n
        - Show the main character (e.g., 'Raju the farmer' or 'Shweta the housewife', etc, whatever is more appropriate) experiencing the challenges described in the scheme's problem statement, such as financial struggles or resource shortages, etc.\n
        - Use a rural or village setting with elements like small farms, simple houses, or crops to illustrate the environment, according to the scheme's objective.\n
        - Focus on conveying the difficulties faced by beneficiaries before receiving support, such as worried expressions, sparse resources, or modest living conditions, etc.
        ///
        {description}
        """
    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that creates image generation prompts."},
            {"role": "user", "content": prompt1}
        ]
    )
    img_prompt1 = completion.choices[0].message.content

    # prompt2 = f"""
    #     Generate an image prompt to illustrate the 'Application Process' for the '{scheme_name}':\n\n"
    #     "{content}\n\n"
    #     "Instructions:\n"
    #     "- Depict a scene showing the enrollment steps, with the main character (e.g., 'Raju the farmer' or 'Shweta the housewife', etc, whatever is more approprite and mentioned later) at a registration desk or using a digital device.\n
    #     "- Highlight the required documents, such as Aadhaar card, bank passbook, or application forms, as visible elements in the scene.\n
    #     "- Include a friendly government official or digital interface to emphasize the assistance provided during the application process.\n
    #     "- Set the scene in a government office or rural setting, focusing on the action of registration and the accessibility of the process.
    #     ///
    #     {description}
    #     ///
    #     Here is the previous image prompt, ensure consistency of characters and story:
    #     {img_prompt1}
    #     """
    prompt2 = f"""
        Generate a short image prompt to illustrate the 'Application Process' for the '{scheme_name}':\n\n"
        "{content}\n\n"
        "Instructions:\n"
        "- Depict a scene showing the enrollment steps, with the main character (e.g., 'Raju the farmer' or 'Shweta the housewife', etc, whatever is more approprite and mentioned later) at a registration desk or using a digital device.\n
        "- Highlight the required documents, such as Aadhaar card, bank passbook, or application forms, as visible elements in the scene.\n
        "- Include a friendly government official or digital interface to emphasize the assistance provided during the application process.\n
        "- Set the scene in a government office or rural setting, focusing on the action of registration and the accessibility of the process.
        ///
        {description}
        """

    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that creates image generation prompts."},
            {"role": "user", "content": prompt2}
        ]
    )
    img_prompt2 = completion.choices[0].message.content

    # prompt3 = f"""
    #     Generate an image prompt for the '{scheme_name}':\n\n"
    #     f"{content}\n\n"
    #     Instructions:\n
    #     - Use a rural/appropriate setting and portray characters benefiting from a government support scheme.
    #     - Include symbols of agricultural aid or community support, illustrating the scheme’s purpose.
    #     ///
    #     {description}
    #     ///
    #     Here are the previous prompts, ensure consistency of characters and story:
    #     {img_prompt1}
    #     {img_prompt2}
    #     """
    prompt3 = f"""
        Generate an short image prompt for the '{scheme_name}':\n\n"
        f"{content}\n\n"
        Instructions:\n
        - Use a rural/appropriate setting and portray characters benefiting from a government support scheme.
        - Include symbols of agricultural aid or community support, illustrating the scheme’s purpose.
        ///
        {description}
        """

    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that creates image generation prompts."},
            {"role": "user", "content": prompt3}
        ]
    )
    img_prompt3 = completion.choices[0].message.content

    return description, img_prompt1, img_prompt2, img_prompt3


### Code snippet to generate questions about generated images
#### PVQ: Prompt Verification Questions (shallower)
#### IBQ: Image Based Questions (deeper)

In [ ]:
import os
from openai import OpenAI

client = OpenAI(api_key = "openai_api")

def get_questions(image_prompt):
    # Prompt Verification Questions = PVQ
    pvq_part1 = f"""Instruction: Given the following image prompt, generate a set of Prompt Verification Questions (PVQ). These questions should focus on confirming whether the generated image matches the description in the prompt. They should verify key elements like characters, actions, objects, and settings mentioned in the prompt.
    Input:
    """
    pvq_part2 = """

    Output Format:
    1. Question
    2. Question
    3. Question
    4. Question
    """

    # Image Based Questions = ibq
    ibq_part1 = f"""Instruction: Given the following image prompt, generate a set of Image-Based Questions (IBQ). These questions should focus on encouraging detailed observation and analysis of the image, covering aspects like specific objects, attire, background elements, or nuanced actions not explicitly stated in the prompt.
    Input:
    """
    ibq_part2 = """

    Output Format:
    1. Question
    2. Question
    3. Question
    4. Question
    """

    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that creates helps verify image generation prompts."},
            {"role": "user", "content": pvq_part1+image_prompt+pvq_part2}
        ]
    )
    pvq = completion.choices[0].message.content

    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that creates helps verify image generation prompts."},
            {"role": "user", "content": ibq_part1+image_prompt+ibq_part2}
        ]
    )
    ibq = completion.choices[0].message.content
    return pvq, ibq

### Helper function to get image provided the image prompt (dall-e-3)

In [ ]:
from openai import OpenAI
client = OpenAI(api_key = 'openai_api')

# original_prompt = new_prompt
def get_image(prompt):
    response = client.images.generate(
    model="dall-e-3",
    prompt=prompt,
    size="1024x1024",
    quality="standard",
    n=1,
    )

    image_url = response.data[0].url
    return image_url

### Code snippet to answer questions about image (gemini-1.5-flash-002)
#### Model takes image and questions as input, and answers them

In [ ]:
import vertexai
from vertexai.generative_models import GenerativeModel, Part
from PIL import Image as PILImage
import io
import os
import requests
import traceback

# Before using this, you need to authenticate yourself in the terminal, will check in google colab
# Replace with your actual project ID
PROJECT_ID = "google_cloud_project"
vertexai.init(project=PROJECT_ID, location="us-central1")


def interpret_image(image_url, pvq, ibq):
    try:
        # Send a GET request to download the image
        response = requests.get(image_url, timeout=10)  # Add timeout to prevent hanging
        response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)

        # Load the image data into a BytesIO object
        img_byte_array = io.BytesIO(response.content)

        # Open the image with PIL
        img = PILImage.open(img_byte_array)

        # Convert the image to byte data (JPEG format)
        final_img_byte_array = io.BytesIO()
        img.save(final_img_byte_array, format='JPEG')
        final_img_byte_array = final_img_byte_array.getvalue()

        # Create a Part object with the byte data
        image_part = Part.from_data(data=final_img_byte_array, mime_type="image/jpeg")

        # Initialize the model
        model = GenerativeModel("gemini-1.5-flash-002")

        # Generate PVQ response
        pvq_response = model.generate_content(
            [
                image_part,  # Pass the image as a part
                pvq,
            ]
        )
        pvq_interpretation = pvq_response.text if hasattr(pvq_response, 'text') else pvq_response

        # Generate IBQ response
        ibq_response = model.generate_content(
            [
                image_part,  # Pass the image as a part
                ibq,
            ]
        )
        ibq_interpretation = ibq_response.text if hasattr(ibq_response, 'text') else ibq_response

        return pvq_interpretation, ibq_interpretation

    except requests.exceptions.RequestException as e:
        print(f"Request error for {image_url}: {e}")
        traceback.print_exc()
    except Exception as e:
        print(f"Error processing {image_url}: {e}")
        traceback.print_exc()

    return None, None

### Helper function to refine prompt according to previous prompt and question-answers on generated image

In [ ]:
from openai import OpenAI
client = OpenAI(api_key = 'openai_api')

def refine_prompt(answer_pvq, answer_ibq, curr_prompt, description):
    image_betterment_prompt = f"""Here is the old image prompt:
    {curr_prompt}
    And here are the answers to some questions about the image generated by the old:
    {answer_pvq}
    {answer_ibq}

    Using the above information, create a better and more detailed version of the old image prompt, but make it @concise@.
    Retain the essence of the original prompt, such as:  'difficulties faced by beneficiaries before receiving support',
    'illustration of the application process', or 'characters benefiting from a government support scheme', whichever one
    of three was used in the above text. Improve its specificity and richness. Space will be left for the original description
    of the characters, and art style, @so do not leave space for it or make the prompt too long@.
    """

    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that creates image generation prompts."},
            {"role": "user", "content": image_betterment_prompt}
        ]
    )
    better_prompt = completion.choices[0].message.content

    Additional_text = f"""Follow the description given below:\n {description}"""

    better_prompt = better_prompt + Additional_text
    return better_prompt


### Function to improve image prompts iteratively

In [ ]:
import time

# Example function to clean the input string (prompt) - helpful to reduce token count
def clean_prompt(prompt):
    # Remove unnecessary sequences or replace them
    prompt = prompt.replace("\\n", "\n")  # Removes escape characters
    prompt = prompt.replace("\\", "")  # Removes escape characters
    prompt = prompt.replace("#", "")  # Removes escape characters
    prompt = prompt.replace("*", "")  # Removes escape characters
    return prompt


def iterative_refinement(iters, image_prompt1, image_prompt2, image_prompt3, description):
    prompts = {
        "prompt1": {},
        "prompt2": {},
        "prompt3": {},
        "url1": {},
        "url2": {},
        "url3": {}
    }

    curr_prompt1 = clean_prompt(image_prompt1)
    curr_prompt2 = clean_prompt(image_prompt2)
    curr_prompt3 = clean_prompt(image_prompt3)

    prompts["prompt1"][0] = curr_prompt1
    prompts["prompt2"][0] = curr_prompt2
    prompts["prompt3"][0] = curr_prompt3

    for i in range(1, iters + 1):
        # Generate image URLs
        url1 = get_image(curr_prompt1)
        time.sleep(5)
        url2 = get_image(curr_prompt2)
        time.sleep(5)
        url3 = get_image(curr_prompt3)
        time.sleep(5)
        print(f"Got images for prompts in iteration {i-1}")


        prompts["url1"][i-1] = url1
        prompts["url2"][i-1] = url2
        prompts["url3"][i-1] = url3

        # Generate PVQ and IBQ
        pvq1, ibq1 = get_questions(curr_prompt1)
        time.sleep(5)
        pvq2, ibq2 = get_questions(curr_prompt2)
        time.sleep(5)
        pvq3, ibq3 = get_questions(curr_prompt3)
        time.sleep(5)
        print(f"Got image questions in iteration {i}")

        # Interpret the images to get answers
        answer_pvq1, answer_ibq1 = interpret_image(url1, pvq1, ibq1)
        answer_pvq2, answer_ibq2 = interpret_image(url2, pvq2, ibq2)
        answer_pvq3, answer_ibq3 = interpret_image(url3, pvq3, ibq3)
        print(f"Got image answers in iteration {i}")

        # Refine the prompts based on answers
        curr_prompt1 = refine_prompt(answer_pvq1, answer_ibq1, curr_prompt1, description)
        time.sleep(5)
        curr_prompt2 = refine_prompt(answer_pvq2, answer_ibq2, curr_prompt2, description)
        time.sleep(5)
        curr_prompt3 = refine_prompt(answer_pvq3, answer_ibq3, curr_prompt3, description)
        time.sleep(5)
        print(f"Refined prompts in iteration {i}")
        print()

        # Clean prompts
        curr_prompt1 = clean_prompt(curr_prompt1)
        curr_prompt2 = clean_prompt(curr_prompt2)
        curr_prompt3 = clean_prompt(curr_prompt3)

        # Store current prompts for this iteration
        prompts["prompt1"][i] = curr_prompt1
        prompts["prompt2"][i] = curr_prompt2
        prompts["prompt3"][i] = curr_prompt3


    # Generate final image URLs
    url1 = get_image(curr_prompt1)
    time.sleep(5)
    url2 = get_image(curr_prompt2)
    time.sleep(5)
    url3 = get_image(curr_prompt3)
    time.sleep(5)
    print(f"Got images for prompts in iteration {iters}")

    prompts["url1"][iters] = url1
    prompts["url2"][iters] = url2
    prompts["url3"][iters] = url3


    return prompts



### Helper functions to save all images and prompts generated during iterative prompt refinement to track progress

In [ ]:
import os
import requests

def save_experiment_images(total_iters, prompts, scheme_name):
    # Create the experiment directory
    exp_dir = "/content/drive/MyDrive/" + "relative path for saving all intermediate images" + scheme_name
    os.makedirs(exp_dir, exist_ok=True)

    # Iterate over each iteration
    for i in range(total_iters + 1):
        # Create iteration-specific subdirectory
        iter_dir = os.path.join(exp_dir, f"Iter{i}")
        os.makedirs(iter_dir, exist_ok=True)

        # Save images for each prompt
        for j in range(1, 4):  # Assuming 3 prompts
            image_url = prompts[f'url{j}'][i]
            image_path = os.path.join(iter_dir, f"image{j}.jpg")

            try:
                # Download the image from the URL
                response = requests.get(image_url, stream=True)
                if response.status_code == 200:
                    # Save the image to the corresponding path
                    with open(image_path, 'wb') as file:
                        file.write(response.content)
                else:
                    print(f"Failed to download image from {image_url}. HTTP status: {response.status_code}")
            except Exception as e:
                print(f"Error downloading image from {image_url}: {e}")

    print('Images saved succesfully')


def save_final_prompts(TOTAL_ITERS, prompts, scheme_name):
    # Create the experiment directory
    exp_dir = "/content/drive/MyDrive/" + "relative path for saving final prompts" + scheme_name
    os.makedirs(exp_dir, exist_ok=True)

    prompt1 = prompts["prompt1"][TOTAL_ITERS]
    prompt2 = prompts["prompt2"][TOTAL_ITERS]
    prompt3 = prompts["prompt3"][TOTAL_ITERS]

    file_path1 = os.path.join(exp_dir, f"{scheme_name}-1.txt")
    with open(file_path1, 'w+') as file:
        file.write(prompt1)

    file_path2 = os.path.join(exp_dir, f"{scheme_name}-2.txt")
    with open(file_path2, 'w+') as file:
        file.write(prompt2)

    file_path3 = os.path.join(exp_dir, f"{scheme_name}-3.txt")
    with open(file_path3, 'w+') as file:
        file.write(prompt3)

    print('Prompt saved succesfully')





### Helper function to save all intermediate prompts in a json format

In [ ]:
import json

def save_all_prompts(prompts, scheme_name):
    experiment_data = {
        "scheme_name": scheme_name,
        "prompts": prompts
    }
    # Write the data to a JSON file
    file_name = "/content/drive/MyDrive/" + "relative path for saving all prompts" + scheme_name +".json"
    with open(file_name, "w+") as f:
        json.dump(experiment_data, f, indent=4)
    print(f"Prompts saved to {file_name}")

## Main function to iteratively refine images for all welfare schemes


In [ ]:
import time

# Start timer
start_time = time.time()

output_directory = "/content/drive/MyDrive/" + "remaining path to folder to save final images"

# Ensure the output directory exists
os.makedirs(output_directory, exist_ok=True)

directory = "/content/drive/MyDrive/" + "remaining path for outputs for LLM write-ups"

file_paths = [os.path.join(directory, file) for file in os.listdir(directory) if file.endswith(".txt")]

results = []


for file_path in file_paths:
    # Remove '.txt' from the file name
    file_name = os.path.basename(file_path)
    scheme_name = file_name.replace(".txt", "")
    description, image_prompt1, image_prompt2, image_prompt3 = get_basic_prompts(file_name, file_path)

    # print(f'\nOriginal description:\n {description}\n\n')

    # Hyperparameter
    TOTAL_ITERS = 4

    print(f"Iterating through prompts for {scheme_name}")
    try:
        prompts = iterative_refinement(TOTAL_ITERS, image_prompt1, image_prompt2, image_prompt3, description)
        print('\n')
        time.sleep(10)

    except Exception as e:
        #Handle API error here, e.g. retry or log
        print(f"OpenAI API returned an API Error: {e} for {scheme_name}")
        continue



    better_prompt1 = prompts["prompt1"][TOTAL_ITERS]
    better_prompt2 = prompts["prompt2"][TOTAL_ITERS]
    better_prompt3 = prompts["prompt3"][TOTAL_ITERS]

    results.append({
        'scheme_name': scheme_name,
        'prompt1': better_prompt1,
        'prompt2': better_prompt2,
        'prompt3': better_prompt3
    })

    # Print all prompts across iterations
    # print("\nAll prompts across iterations:")
    # for i in range(0, TOTAL_ITERS + 1):
    #     print(f"Iteration {i}:")
    #     print(f"  Prompt 1: {prompts['prompt1'][i]}")
    #     print(f"  Prompt 2: {prompts['prompt2'][i]}")
    #     print(f"  Prompt 3: {prompts['prompt3'][i]}")
    #     print('-'* 80)
    #     print('')


    # Print all prompts across iterations
    # print("\n\nAll links across iterations:")
    # for i in range(0, TOTAL_ITERS + 1):
    #     print(f"Iteration {i}:")
    #     print(f"  Link 1: {prompts['url1'][i]}")
    #     print(f"  Link 2: {prompts['url2'][i]}")
    #     print(f"  Link 3: {prompts['url3'][i]}")
    #     print('-'* 80)
    #     print('')

    save_experiment_images(TOTAL_ITERS, prompts, scheme_name)
    save_final_prompts(TOTAL_ITERS, prompts, scheme_name)
    save_all_prompts(prompts, scheme_name)

    for j in range(1, 4):
        image_url = prompts[f'url{j}'][TOTAL_ITERS]
        image_path = os.path.join(output_directory, f"{scheme_name}_{j}.jpg")
        try:
            # Download the image from the URL
            response = requests.get(image_url, stream=True)
            if response.status_code == 200:
                # Save the image to the corresponding path
                with open(image_path, 'wb') as file:
                    file.write(response.content)
            else:
                print(f"Failed to download image from {image_url}. HTTP status: {response.status_code}")
        except Exception as e:
            print(f"Error downloading image from {image_url}: {e}")


# Once the loop finishes, write the results dictionary to a text file
results_file_path = "/content/drive/MyDrive/" + "relative path" + "results_2.txt"
with open(results_file_path, 'w+') as results_file:
    results_file.write(str(results))

print(f"Results written to '{results_file_path}'")

# End timer
end_time = time.time()

# Calculate and print execution time
execution_time = end_time - start_time
minutes, seconds = divmod(execution_time, 60)  # Converts to minutes and seconds
print(f"Execution Time: {int(minutes)} minutes and {seconds:.2f} seconds")

### Code to check quality of image prompts
#### (Executing the cell below is not necessary)

In [ ]:
!pip install textstat
import os
import json
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoTokenizer, AutoModel
import torch
import textstat
from glob import glob

# Load a transformer model for embeddings
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

def read_prompts_from_json(file_path):
    """Reads prompts from a JSON file, ignoring URL entries."""
    with open(file_path, "r") as file:
        data = json.load(file)

    # Extract all iterations of prompts1, prompts2, prompts3
    prompts = []
    for prompt_key in ["prompt1", "prompt2", "prompt3"]:
        if prompt_key in data["prompts"]:
            iterations = data["prompts"][prompt_key]
            for iteration in sorted(iterations.keys(), key=lambda x: int(x)):  # Sort by numeric keys
                prompts.append(iterations[iteration])
    return prompts

def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).detach().numpy().reshape(1, -1)

def calculate_cosine_similarity(prompts):
    embeddings = [get_embedding(prompt) for prompt in prompts]
    similarities = []
    for i in range(len(embeddings)):
        for j in range(i + 1, len(embeddings)):
            similarities.append(cosine_similarity(embeddings[i], embeddings[j])[0][0])
    avg_similarity = np.mean(similarities) if similarities else 0
    return avg_similarity

def calculate_token_lengths(prompts):
    return [len(prompt.split()) for prompt in prompts]

def calculate_lexical_diversity(prompts):
    return [len(set(prompt.split())) / len(prompt.split()) for prompt in prompts]

def calculate_readability_scores(prompts):
    flesch_scores = [textstat.flesch_reading_ease(prompt) for prompt in prompts]
    fog_scores = [textstat.gunning_fog(prompt) for prompt in prompts]
    return flesch_scores, fog_scores

def evaluate_prompts_for_scheme(scheme_name, file_path):
    prompts = read_prompts_from_json(file_path)

    # Consistency metrics
    avg_cosine_similarity = calculate_cosine_similarity(prompts)
    token_lengths = calculate_token_lengths(prompts)
    lexical_diversity = calculate_lexical_diversity(prompts)
    flesch_scores, fog_scores = calculate_readability_scores(prompts)

    # Display metrics for the scheme
    print(f"Metrics for Scheme: {scheme_name}")
    print(f"Average Cosine Similarity: {avg_cosine_similarity:.4f}")
    print("Token Lengths:", token_lengths)
    print("Lexical Diversity:", [f"{diversity:.4f}" for diversity in lexical_diversity])
    print("Readability Scores (Flesch Reading Ease):", flesch_scores)
    print("Readability Scores (Gunning Fog):", fog_scores)
    print("-" * 60)

def main(directory_path):
    # Find all JSON files in the directory
    file_paths = glob(os.path.join(directory_path, "*.json"))

    for file_path in file_paths:
        # Extract the scheme name from the filename (excluding extension)
        scheme_name = os.path.splitext(os.path.basename(file_path))[0]
        evaluate_prompts_for_scheme(scheme_name, file_path)

# Define the directory where all prompt files are stored
directory_path = "/content/drive/MyDrive/" + "remaining path of the new image prompts"

# Run the evaluation
main(directory_path)